In [1]:
import pandas as pd
import re

list_data_csv = pd.read_csv('input/input.csv')
list_data_csv


,address
0,東京都大田区大森本町1-8-12-S-1203
1,東京都大田区山王1-7-8山王薬局1丁目店
2,東京都台東区浅草橋1-32-6-2F
3,東京都品川区高輪4-8-10-5階
4,東京都大田区西六郷4-30-7音金六郷マンション1-C号
...,...
57,名古屋市熱田区神宮四丁目７番２７号
58,名古屋市熱田区神宮四丁目７番２７号Bビル
59,名古屋市港区空見町36番地
60,名古屋市港区十一屋3丁目63番地


In [2]:
type(list_data_csv['address'][0])

str

In [3]:
def find_prefacture(string: str):
    # 北海道,大阪府,京都府,東京都は別扱い
    special_prefactures: list = ["北海道","大阪府","京都府","東京都"]
    for _ in special_prefactures:
        if _ in string:
            return (_,)

def erase_special_address_expression(string: str, index: int):
    size: int = len(string)
    # 丁目を置換することのみ行う
    if index == 1:
        res: str = ""
        i :int = 0
        while i < (size):
            if i == size-1:
                res = res + string[i]
                i += 1
            else:
                if(string[i]=='丁' and string[i+1]=='目'):
                    # 丁目店などは特殊ケースとして除外
                    if i+2 < size and string[i+2] == "店":
                        res = res + string[i]
                        i += 1
                        continue
                    res = res + ("-")
                    i+=2
                else: 
                    res = res + (string[i])
                    i += 1
        return res
    # 番地を置換することのみ行う
    if index == 2:
        res: str = ""
        i :int = 0
        while i < size:
            if i == size-1:
                res = res + string[i]
                i += 1
            else:
                if(string[i]=="番" and string[i+1]=="地"):
                    res = res + ("-")
                    i+=2
                else: 
                    res = res + (string[i])
                    i+=1
        return res
    # 番を置換することのみ行う
    if index == 3:
        res: str = ""
        for i in range(size):
                if(string[i]=="番"): res = res + ("-")
                else: res = res + (string[i])
        return res
    # の を置換する
    if index == 4:
        res: str = ""

        numbers: list = []
        for i in range(10):
            numbers.append(str(i))
        
        for i in range(size):
            if i > 0 and i+1 < size and string[i]=="の" and string[i-1] in numbers:
                res = res + "-"
            else: res = res + string[i]
        return res
    if index == 5:
        return string.translate(str.maketrans({chr(0xFF01 + i): chr(0x21 + i) for i in range(94)}))
        # # 全角のアルファベットを半角に変換
        # alphabet: list = []
        # mapping_alphabet: dict = {}
        # for i in range(26):
        #     alphabet.append(chr(i+ord('A')))
        #     mapping_alphabet[chr(i+ord('A'))] = chr(i+ord('A'))
        # res: str = ""
        # for i in range(size):
        #     if string[i] in alphabet:
        #         res = res + mapping_alphabet[string[i]]
        #     else:
        #         res = res + string[i]
        # return res


    
def  replace_japanese_address_expression(string: str):
    # ~丁目,~番,~番地を-に変換
    # 注意: 番地 -> 番 の順番に行わないとデータが壊れる
    if "丁目" in string:
        string = erase_special_address_expression(string, 1)
    if "番地" in string:
        string = erase_special_address_expression(string, 2)
    if "番" in string:
        string = erase_special_address_expression(string, 3)
    if "の" in string:
        string = erase_special_address_expression(string, 4)
    string = erase_special_address_expression(string, 5)
    return string

In [4]:
replaced_address_data: list = []

def erase_last_hyphen(string: str):
    if string[-1]=="-":
        return string[0:len(string)-1]
    else: return string

def replace_slash_with_hyphen(string: str):
    # 空白を取り除く
    string = string.replace(' ', '-')
    string = string.replace('　', '-')
    # 正規表現で/を-に置き換える
    return re.sub("/", "-", string)

def japanese_style_number_to_number(string: str):
    def function(char: str):
        dictionary_japanese_style_number: dict = {"一":"1","二":"2","三":"3","四":"4","五":"5","六":"6","七":"7","八":"8","九":"9","十":"10","〇":"0"}
        return dictionary_japanese_style_number[char]
    def full_width_to_half_width(char :str):
        mapping_dictionary: dict = {"１":"1","２":"2","３":"3","４":"4","５":"5","６":"6","７":"7","８":"8","９":"9","０":"10"}
        full_width_number: list = ["１","２","３","４","５","６","７","８","９","０"]
        if char in full_width_number:
            return mapping_dictionary[c]
        else: return char
    japanese_style_number: list = ["一","二","三","四","五","六","七","八","九","〇"]
    # 十と表記する際は高確率で地名なので
    
    res: str = ""
    for c in string:
        if c in japanese_style_number:
            res = res + full_width_to_half_width(function(c))
        else: res = res + full_width_to_half_width(c)
    return res

def full_width_hyphen_to_half_width_hyphen(string: str):
    res: str = ""
    for char in string:
        if char == "ー":
            res = res + "-"
        else: res = res + char
    return res

def operation(string:str):
    return full_width_hyphen_to_half_width_hyphen(japanese_style_number_to_number(replace_slash_with_hyphen(erase_last_hyphen(replace_japanese_address_expression(string)))))

for address_data in list_data_csv['address']:
    replaced_address_data.append(operation(address_data))

pd.DataFrame(replaced_address_data)



,0
0,東京都大田区大森本町1-8-12-S-1203
1,東京都大田区山王1-7-8山王薬局1丁目店
2,東京都台東区浅草橋1-32-6-2F
3,東京都品川区高輪4-8-10-5階
4,東京都大田区西6郷4-30-7音金6郷マンション1-C号
...,...
57,名古屋市熱田区神宮4-7-27号
58,名古屋市熱田区神宮4-7-27号Bビル
59,名古屋市港区空見町36
60,名古屋市港区十1屋3-63


In [5]:
data: dict = {}
data['original'] = list_data_csv['address']

def extract_prefacture(string: str):
    prefactures: list = ["北海道","青森県","岩手県","宮城県","秋田県","山形県","福島県","茨城県","栃木県","群馬県","埼玉県","千葉県","東京都","神奈川県","新潟県","富山県","石川県","福井県","山梨県","長野県","岐阜県","静岡県","愛知県","三重県","滋賀県","京都府","大阪府","兵庫県","奈良県","和歌山県","鳥取県","島根県","岡山県","広島県","山口県","徳島県","香川県","愛媛県","高知県","福岡県","佐賀県","長崎県","熊本県","大分県","宮崎県","鹿児島県","沖縄県"]
    
    for prefacture in prefactures:
        if prefacture in string:
            return (prefacture,string.replace(prefacture, ""))
    # 県名が存在しない
    return ("", string)

prefactures: list = []
non_prefacture_address_data: list = []
for string in replaced_address_data:
    tuple_data: tuple = extract_prefacture(string)
    prefactures.append(tuple_data[0])
    non_prefacture_address_data.append(tuple_data[1])

data["prefacture"] = prefactures
# data["other"] = non_prefacture_address_data

pd.DataFrame(data)

,original,prefacture
0,東京都大田区大森本町1-8-12-S-1203,東京都
1,東京都大田区山王1-7-8山王薬局1丁目店,東京都
2,東京都台東区浅草橋1-32-6-2F,東京都
3,東京都品川区高輪4-8-10-5階,東京都
4,東京都大田区西六郷4-30-7音金六郷マンション1-C号,東京都
...,...,...
57,名古屋市熱田区神宮四丁目７番２７号,
58,名古屋市熱田区神宮四丁目７番２７号Bビル,
59,名古屋市港区空見町36番地,
60,名古屋市港区十一屋3丁目63番地,


In [6]:
non_prefacture_address_data
# ここから市、区を取り除く

cities: list = []
non_city_address_data: list = []

for city_address_data in non_prefacture_address_data:
    if "市" in city_address_data:
        index: int = city_address_data.find("市")
        cities.append(city_address_data[0:index+1])
        if len(city_address_data) > index+1:
            # index out of rangeを避けるため
            non_city_address_data.append(city_address_data[index+1:])
    elif "区" in city_address_data:
        index: int = city_address_data.find("区")
        cities.append(city_address_data[0:index+1])
        if len(city_address_data) > index+1:
            # index out of rangeを避けるため
            non_city_address_data.append(city_address_data[index+1:])
    elif "郡" in city_address_data:
        index: int = city_address_data.find("郡")
        cities.append(city_address_data[0:index+1])
        if len(city_address_data) > index+1:
            # index out of rangeを避けるため
            non_city_address_data.append(city_address_data[index+1:])
    elif "町" in city_address_data:
        index: int = city_address_data.find("町")
        cities.append(city_address_data[0:index+1])
        if len(city_address_data) > index+1:
            # index out of rangeを避けるため
            non_city_address_data.append(city_address_data[index+1:])
    elif "村" in city_address_data:
        index: int = city_address_data.find("村")
        cities.append(city_address_data[0:index+1])
        if len(city_address_data) > index+1:
            # index out of rangeを避けるため
            non_city_address_data.append(city_address_data[index+1:])
    else:
        cities.append("")
        non_city_address_data.append(city_address_data)

data['city'] = cities
# data['other'] = non_city_address_data


pd.DataFrame(data)

,original,prefacture,city
0,東京都大田区大森本町1-8-12-S-1203,東京都,大田区
1,東京都大田区山王1-7-8山王薬局1丁目店,東京都,大田区
2,東京都台東区浅草橋1-32-6-2F,東京都,台東区
3,東京都品川区高輪4-8-10-5階,東京都,品川区
4,東京都大田区西六郷4-30-7音金六郷マンション1-C号,東京都,大田区
...,...,...,...
57,名古屋市熱田区神宮四丁目７番２７号,,名古屋市
58,名古屋市熱田区神宮四丁目７番２７号Bビル,,名古屋市
59,名古屋市港区空見町36番地,,名古屋市
60,名古屋市港区十一屋3丁目63番地,,名古屋市


In [7]:
non_city_address_data
# ここからさらに小さな行政単位を取り除く

towns: list = []
non_town_address_data: list = []

for town_address_data in non_city_address_data:
    if "区" in town_address_data:
        index: int = town_address_data.find("区")
        towns.append(town_address_data[0:index+1])
        if len(city_address_data) > index+1:
            # index out of rangeを避けるため
            non_town_address_data.append(town_address_data[index+1:])
    elif "郡" in town_address_data:
        index: int = town_address_data.find("郡")
        towns.append(town_address_data[0:index+1])
        if len(town_address_data) > index+1:
            # index out of rangeを避けるため
            non_town_address_data.append(town_address_data[index+1:])
    elif "町" in town_address_data:
        index: int = town_address_data.find("町")
        towns.append(town_address_data[0:index+1])
        if len(town_address_data) > index+1:
            # index out of rangeを避けるため
            non_town_address_data.append(town_address_data[index+1:])
    elif "村" in town_address_data:
        index: int = town_address_data.find("村")
        towns.append(town_address_data[0:index+1])
        if len(town_address_data) > index+1:
            # index out of rangeを避けるため
            non_town_address_data.append(town_address_data[index+1:])
    else:
        towns.append("")
        non_town_address_data.append(town_address_data)

data['town'] = towns
# data['other'] = non_town_address_data


pd.DataFrame(data)
# city town districtに数字か-があったら要注意

,original,prefacture,city,town
0,東京都大田区大森本町1-8-12-S-1203,東京都,大田区,大森本町
1,東京都大田区山王1-7-8山王薬局1丁目店,東京都,大田区,
2,東京都台東区浅草橋1-32-6-2F,東京都,台東区,
3,東京都品川区高輪4-8-10-5階,東京都,品川区,
4,東京都大田区西六郷4-30-7音金六郷マンション1-C号,東京都,大田区,
...,...,...,...,...
57,名古屋市熱田区神宮四丁目７番２７号,,名古屋市,熱田区
58,名古屋市熱田区神宮四丁目７番２７号Bビル,,名古屋市,熱田区
59,名古屋市港区空見町36番地,,名古屋市,港区
60,名古屋市港区十一屋3丁目63番地,,名古屋市,港区


In [8]:
non_town_address_data

districts: list = []
others: list = []

def extract_district(string: str):
    district_numbers:list = []
    for i in range(0,10):
        district_numbers.append(str(i))
    # 数字が現れるまでを記録する
    for i in range(len(string)):
        if string[i] in district_numbers:
            return (string[0:i],string[i:])
    return ("",string)
        
for district_data in non_town_address_data:
    district_tuple_data = extract_district(district_data)
    districts.append(district_tuple_data[0])
    others.append(district_tuple_data[1])

data['district'] = districts
# data['others'] = others
pd.set_option("display.max_rows",100)
pd.DataFrame(data)

# 全角の2が混ざっている

,original,prefacture,city,town,district
0,東京都大田区大森本町1-8-12-S-1203,東京都,大田区,大森本町,
1,東京都大田区山王1-7-8山王薬局1丁目店,東京都,大田区,,山王
2,東京都台東区浅草橋1-32-6-2F,東京都,台東区,,浅草橋
3,東京都品川区高輪4-8-10-5階,東京都,品川区,,高輪
4,東京都大田区西六郷4-30-7音金六郷マンション1-C号,東京都,大田区,,西
5,北海道蛭田郡豊浦町字桜207,北海道,蛭田郡,豊浦町,字桜
6,東京都千代田区一番町8,東京都,千代田区,1-町,
7,豊平区豊平4条3-3-10-403,,豊平区,,豊平
8,住吉1-17-18 アルぺ-ジュ401,,,,住吉
9,本町田2428-ハ-10-109,,本町,,田


In [9]:
others
# ここから1-3-5などを抜き取る
house_numbers: list = []
others_head: list = []
others_tail: list = []

def extract_house_number(string: str):
    index = re.search('(([0-9]+)-)+([0-9]+)',string)
    if index == None:
        house_numbers.append("")
        others_head.append("")
        others_tail.append(string)
    else:
        house_numbers.append(string[index.start():index.end()])
        others_head.append(string[0:index.start()])
        others_tail.append(string[index.end():])
        

for other in others:
    extract_house_number(other)

data["invalid"] =  others_head
data["house_number"] = house_numbers
# data["others_tail"] = others_tail

def check_vaild_word_or_not(index: int):
    if re.search('^([0-9])+([ぁ-んァ-ヶｱ-ﾝﾞﾟ一-龠])+$',data['invalid'][index]) != None:
        mapping_arithmetic_number_to_japanese_number: dict = {'1':'一','2':'二','3':'三','4':'四','5':'五','6':'六','7':'七','8':'八','9':'九'}
        numbers: list = [str(i) for i in range(1,10)]
        # print(numbers)
        res: str = ""
        for char in data['invalid'][index]:
            if(char in numbers):
                res = res + mapping_arithmetic_number_to_japanese_number[char]
            else: res = res + char
        return res
    else: return ""

cation: list = []
for index in range(len(others_head)):
    response: str = check_vaild_word_or_not(index)
    if data['invalid'][index]=="":
        cation.append(False)
    elif response == "":
        # 不正な文字列
        cation.append(True)
    else:
        cation.append(False)
        data['invalid'][index] = ""
        data['district'][index] += response

pd.DataFrame(data)

,original,prefacture,city,town,district,invalid,house_number
0,東京都大田区大森本町1-8-12-S-1203,東京都,大田区,大森本町,,,1-8-12
1,東京都大田区山王1-7-8山王薬局1丁目店,東京都,大田区,,山王,,1-7-8
2,東京都台東区浅草橋1-32-6-2F,東京都,台東区,,浅草橋,,1-32-6-2
3,東京都品川区高輪4-8-10-5階,東京都,品川区,,高輪,,4-8-10-5
4,東京都大田区西六郷4-30-7音金六郷マンション1-C号,東京都,大田区,,西六郷,,4-30-7
5,北海道蛭田郡豊浦町字桜207,北海道,蛭田郡,豊浦町,字桜,,
6,東京都千代田区一番町8,東京都,千代田区,1-町,,,
7,豊平区豊平4条3-3-10-403,,豊平区,,豊平四条,,3-3-10-403
8,住吉1-17-18 アルぺ-ジュ401,,,,住吉,,1-17-18
9,本町田2428-ハ-10-109,,本町,,田,2428-ハ-,10-109


In [10]:
# 数字が other_tailにあるのでそれを補完
# 名前のところが - になっている, でもそうでないとうまく分類分けできない

others_tail
munipulated_others_tail: list = []

for id in range(len(others_tail)):
    if re.match('[0-9]+$',others_tail[id]):
        print(others_tail[id])
        data['house_number'][id] = data['house_number'][id] + others_tail[id]
        munipulated_others_tail.append("")
    else:
        munipulated_others_tail.append(others_tail[id])

# data['others_tail'] = munipulated_others_tail
pd.DataFrame(data)



207
8
1
86
36


,original,prefacture,city,town,district,invalid,house_number
0,東京都大田区大森本町1-8-12-S-1203,東京都,大田区,大森本町,,,1-8-12
1,東京都大田区山王1-7-8山王薬局1丁目店,東京都,大田区,,山王,,1-7-8
2,東京都台東区浅草橋1-32-6-2F,東京都,台東区,,浅草橋,,1-32-6-2
3,東京都品川区高輪4-8-10-5階,東京都,品川区,,高輪,,4-8-10-5
4,東京都大田区西六郷4-30-7音金六郷マンション1-C号,東京都,大田区,,西六郷,,4-30-7
5,北海道蛭田郡豊浦町字桜207,北海道,蛭田郡,豊浦町,字桜,,207
6,東京都千代田区一番町8,東京都,千代田区,1-町,,,8
7,豊平区豊平4条3-3-10-403,,豊平区,,豊平四条,,3-3-10-403
8,住吉1-17-18 アルぺ-ジュ401,,,,住吉,,1-17-18
9,本町田2428-ハ-10-109,,本町,,田,2428-ハ-,10-109


In [11]:
def extract_number_and_alphabet1(string: str):
    if(string==""): return None
    return re.match('^(-[A-Z 0-9]+)*([A-Z 0-9]+)*$', string)

def extract_number_and_alphabet2(string: str):
    if(string==""): return None
    return re.match('^([A-Z 0-9]+)*(-[A-Z 0-9]+)*$',string)

special_characters: list = []
for i in range(len(munipulated_others_tail)):
    if extract_number_and_alphabet1(munipulated_others_tail[i]):
        special_characters.append(munipulated_others_tail[i])
        munipulated_others_tail[i] = ""
    else: special_characters.append("")
    if extract_number_and_alphabet2(munipulated_others_tail[i]):
        special_characters[i] = (munipulated_others_tail[i])
        munipulated_others_tail[i] = ""
    else: pass

data['special_characters'] = special_characters
def extract_number_from_others_tail(string: str):
    if string=="": return None
    else:
        return re.match('^([0-9]+)',string)
# def extract_number_and_hypen_from_others_tail(string: str):
#     if string=="": return None
#     else:
#         return re.match('^(([0-9]-)+)',string)

for i in range(len(munipulated_others_tail)):
    # if extract_number_and_hypen_from_others_tail(munipulated_others_tail[i]):
    #     start: int = extract_number_and_hypen_from_others_tail(munipulated_others_tail[i]).start()
    #     end: int = extract_number_and_hypen_from_others_tail(munipulated_others_tail[i]).end()
    #     if start != 0: print('something wrong') # for debug
    #     data['house_number'][i] = data['house_number'][i] + munipulated_others_tail[i][start:end]
    #     munipulated_others_tail[i] = munipulated_others_tail[i][end:]
    # else: pass
    if extract_number_from_others_tail(munipulated_others_tail[i]):
        start: int = extract_number_from_others_tail(munipulated_others_tail[i]).start()
        end: int = extract_number_from_others_tail(munipulated_others_tail[i]).end()
        if start != 0: print('something wrong') # for debug
        data['house_number'][i] = data['house_number'][i] + munipulated_others_tail[i][start:end]
        munipulated_others_tail[i] = munipulated_others_tail[i][end:]
    else: pass

# others_tailの整形
def eliminate_hypen(string: str):
    if string == "": return string
    else:
        if string[0]=="-":
            return string[1:]
        else: return string
for i in range(len(munipulated_others_tail)):
    munipulated_others_tail[i] = eliminate_hypen(munipulated_others_tail[i])

# デバッグ用
# data['others_tail'] = munipulated_others_tail
pd.DataFrame(data)

,original,prefacture,city,town,district,invalid,house_number,special_characters
0,東京都大田区大森本町1-8-12-S-1203,東京都,大田区,大森本町,,,1-8-12,-S-1203
1,東京都大田区山王1-7-8山王薬局1丁目店,東京都,大田区,,山王,,1-7-8,
2,東京都台東区浅草橋1-32-6-2F,東京都,台東区,,浅草橋,,1-32-6-2,F
3,東京都品川区高輪4-8-10-5階,東京都,品川区,,高輪,,4-8-10-5,
4,東京都大田区西六郷4-30-7音金六郷マンション1-C号,東京都,大田区,,西六郷,,4-30-7,
5,北海道蛭田郡豊浦町字桜207,北海道,蛭田郡,豊浦町,字桜,,207,
6,東京都千代田区一番町8,東京都,千代田区,1-町,,,8,
7,豊平区豊平4条3-3-10-403,,豊平区,,豊平四条,,3-3-10-403,
8,住吉1-17-18 アルぺ-ジュ401,,,,住吉,,1-17-18,
9,本町田2428-ハ-10-109,,本町,,田,2428-ハ-,10-109,


In [12]:
# special_chractersのなかの F のみ抽出
building_detail_info: list = []

def find_F(string: str):
    '''special_charactersに存在するFという文字単体を検知する'''
    if string == 'F':
        return True
    else: return False

def cutting_number_from_last(index: int):
    '''上記の関数と同時に使用する。8Fのように、建物の階数情報のみを抽出する'''
    if re.search('-([0-9]+)$',data['house_number'][index]) == None:
        print(re.search('-([0-9]+)$',data['house_number'][index]))
        print(data['house_number'][index])
        print('something wrong1') # for debug
    else:
        start: int = re.search('-([0-9]+)$',data['house_number'][index]).start()
        end: int = re.search('-([0-9]+)$',data['house_number'][index]).end()
        if end != len(data['house_number'][index]): print('somethin wrong2') # for debug
        # start+1 にしているのは-{数字}Fとなっているので - を除いている
        building_detail_info.append(data['house_number'][index][start+1:end] + 'F')
        data['house_number'][index] = data['house_number'][index][:start]

for i in range(len(data['special_characters'])):
    if(find_F(data['special_characters'][i])):
        # 空白に変える
        data['special_characters'][i] = ""
        # special_charactersから数字をfetch
        cutting_number_from_last(i)
    else: building_detail_info.append('')

data['building_detail_info'] = building_detail_info

def find_kai(string: str):
    '''階という文字単体が配列munipulated_others_tailに存在する場合検知する'''
    if string=="階":
        return True
    else: return False

def cutting_number_from_last2(index: int):
    '''上記の関数と同時に使用する。8階のように、建物の階数情報のみを抽出する'''
    if re.search('-([0-9]+)$',data['house_number'][index]) == None:
        print(re.search('-([0-9]+)$',data['house_number'][index]))
        print(data['house_number'][index])
        print('something wrong1') # for debug
    else:
        start: int = re.search('-([0-9]+)$',data['house_number'][index]).start()
        end: int = re.search('-([0-9]+)$',data['house_number'][index]).end()
        if end != len(data['house_number'][index]): print('somethin wrong2') # for debug
        # start+1 にしているのは-{数字}階となっているので - を除いている
        data['building_detail_info'][index] = (data['house_number'][index][start+1:end] + '階')
        data['house_number'][index] = data['house_number'][index][:start]

for i in range(len(munipulated_others_tail)):
    if(find_kai(munipulated_others_tail[i])):
        # 空白に変える
        munipulated_others_tail[i] = ""
        # special_charactersから数字をfetch
        cutting_number_from_last2(i)
    else: pass

def find_gou(string: str):
    '''号という文字単体が配列munipulated_others_tailに存在する場合検知する'''
    if string=="号":
        return True
    else: return False

def cutting_number_from_last3(index: int):
    '''上記の関数と同時に使用する。8号のように、建物の階数情報のみを抽出する'''
    if re.search('-([0-9]+)$',data['house_number'][index]) == None:
        print(re.search('-([0-9]+)$',data['house_number'][index]))
        print(data['house_number'][index])
        print('something wrong1') # for debug
    else:
        start: int = re.search('-([0-9]+)$',data['house_number'][index]).start()
        end: int = re.search('-([0-9]+)$',data['house_number'][index]).end()
        if end != len(data['house_number'][index]): print('somethin wrong2') # for debug
        # start+1 にしているのは-{数字}階となっているので - を除いている
        data['building_detail_info'][index] = (data['house_number'][index][start+1:end] + '号')
        data['house_number'][index] = data['house_number'][index][:start]

for i in range(len(munipulated_others_tail)):
    if(find_gou(munipulated_others_tail[i])):
        # 空白に変える
        munipulated_others_tail[i] = ""
        # special_charactersから数字をfetch
        cutting_number_from_last3(i)
    else: pass


In [13]:
# '-(([0-9 A-Z])+)号$' のような情報を抽出する

def extract_building_detail_info_from_others_tail_1(index: int):
    '''501号のようなかたちを検出しbuilding_detail_infoにデータを付け替える'''
    if re.search('[0-9]+号$',munipulated_others_tail[index])!=None:
        start: int = re.search('[0-9]+号$',munipulated_others_tail[index]).start()
        end: int = re.search('[0-9]+号$',munipulated_others_tail[index]).end()
        # ここ順番注意
        building_info = munipulated_others_tail[index][start:end]
        munipulated_others_tail[index] = munipulated_others_tail[index][:start]
        print(building_info)
        # 順番に注意
        data['building_detail_info'][index] = building_info + data['building_detail_info'][index]
    else: pass

def extract_building_detail_info_from_others_tail_2(index: int):
    '''401のようなかたちを検出しbuilding_detail_infoにデータを付け替える'''
    if re.search('[0-9]+$',munipulated_others_tail[index])!=None:
        start: int = re.search('[0-9]+$',munipulated_others_tail[index]).start()
        end: int = re.search('[0-9]+$',munipulated_others_tail[index]).end()
        # ここ順番注意
        building_info = munipulated_others_tail[index][start:end]
        munipulated_others_tail[index] = munipulated_others_tail[index][:start]
        print(building_info)
        # 順番に注意
        data['building_detail_info'][index] = building_info + data['building_detail_info'][index]
    else: pass

def extract_building_detail_info_from_others_tail_3(index: int):
    '''45号館のようなかたちを検出しbuilding_detail_infoにデータを付け替える'''
    if re.search('[0-9]+号館$',munipulated_others_tail[index])!=None:
        start: int = re.search('[0-9]+号館$',munipulated_others_tail[index]).start()
        end: int = re.search('[0-9]+号館$',munipulated_others_tail[index]).end()
        # ここ順番注意
        building_info = munipulated_others_tail[index][start:end]
        munipulated_others_tail[index] = munipulated_others_tail[index][:start]
        print(building_info)
        # 順番に注意
        data['building_detail_info'][index] = building_info + data['building_detail_info'][index]
    else: pass

def extract_building_detail_info_from_others_tail_4(index: int):
    '''C号のようなかたちを検出しbuilding_detail_infoにデータを付け替える'''
    if re.search('-[A-Z]号$',munipulated_others_tail[index])!=None:
        start: int = re.search('-[A-Z]号$',munipulated_others_tail[index]).start()
        end: int = re.search('-[A-Z]号$',munipulated_others_tail[index]).end()
        # ここ順番注意
        building_info = munipulated_others_tail[index][start:end]
        munipulated_others_tail[index] = munipulated_others_tail[index][:start]
        print(building_info)
        # 順番に注意
        data['building_detail_info'][index] = building_info + data['building_detail_info'][index]
    else: pass

def extract_building_detail_info_from_others_tail_5(index: int):
    '''C号のようなかたちを検出しbuilding_detail_infoにデータを付け替える'''
    if re.search('[A-Z]号$',munipulated_others_tail[index])!=None:
        start: int = re.search('[A-Z]号$',munipulated_others_tail[index]).start()
        end: int = re.search('[A-Z]号$',munipulated_others_tail[index]).end()
        # ここ順番注意
        building_info = munipulated_others_tail[index][start:end]
        munipulated_others_tail[index] = munipulated_others_tail[index][:start]
        print(building_info)
        # 順番に注意
        data['building_detail_info'][index] = building_info + data['building_detail_info'][index]
    else: pass

for index in range(len(munipulated_others_tail)):
    extract_building_detail_info_from_others_tail_1(index)
    extract_building_detail_info_from_others_tail_2(index)
    extract_building_detail_info_from_others_tail_3(index)
    extract_building_detail_info_from_others_tail_4(index)
    extract_building_detail_info_from_others_tail_5(index)


-C号
401
205
501号
15
1号


In [14]:
# others_tail内部の半角-を全角に直す
def replace_half_hypen_with_full_width_hypen(string: str):
    res: str = ""
    for char in string:
        if char == '-': res = res + 'ー'
        else: res = res + char
    return res

for i in range(len(munipulated_others_tail)):
    munipulated_others_tail[i] = replace_half_hypen_with_full_width_hypen(munipulated_others_tail[i])

data['building_info'] = munipulated_others_tail
data['building_detail_info'] = data['building_detail_info']
data['cation'] = cation

In [15]:
# cationについて
for index in range(len(data['building_info'])):
    if re.search('(^号)|(^号館)',data["building_info"][index]) != None:
        data['cation'][index] = True
    else: pass
# 不正なデータを検出

In [16]:
df = pd.DataFrame(data)
df = df.reindex(columns=['original','prefacture','city','town','district','invalid','house_number','special_characters','building_info','building_detail_info','cation'])
df

,original,prefacture,city,town,district,invalid,house_number,special_characters,building_info,building_detail_info,cation
0,東京都大田区大森本町1-8-12-S-1203,東京都,大田区,大森本町,,,1-8-12,-S-1203,,,False
1,東京都大田区山王1-7-8山王薬局1丁目店,東京都,大田区,,山王,,1-7-8,,山王薬局1丁目店,,False
2,東京都台東区浅草橋1-32-6-2F,東京都,台東区,,浅草橋,,1-32-6,,,2F,False
3,東京都品川区高輪4-8-10-5階,東京都,品川区,,高輪,,4-8-10,,,5階,False
4,東京都大田区西六郷4-30-7音金六郷マンション1-C号,東京都,大田区,,西六郷,,4-30-7,,音金6郷マンション1,-C号,False
5,北海道蛭田郡豊浦町字桜207,北海道,蛭田郡,豊浦町,字桜,,207,,,,False
6,東京都千代田区一番町8,東京都,千代田区,1-町,,,8,,,,False
7,豊平区豊平4条3-3-10-403,,豊平区,,豊平四条,,3-3-10-403,,,,False
8,住吉1-17-18 アルぺ-ジュ401,,,,住吉,,1-17-18,,アルぺージュ,401,False
9,本町田2428-ハ-10-109,,本町,,田,2428-ハ-,10-109,,,,True


In [17]:
df.to_csv('output/output.csv',encoding='utf-8_sig')
# 文字化けについて https://qiita.com/y4m3/items/674423b596284bbc7cf7
